<a href="https://colab.research.google.com/github/angelohafner/linguagem-de-programacao-udesc/blob/main/substituicao_tags_microsoft_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.0 MB/s eta 0:00:00


In [ ]:
# Requirements: python-docx (pip install python-docx)
from docx import Document
from docx.oxml.text.paragraph import CT_P
from docx.table import _Cell, Table

# --- User settings (use raw strings on Windows) ---
input_path = "/content/Petro_Tags_Template.docx"
output_path = "/content/Petro_Tags_Filled.docx"

# --- Tag mapping (all numeric values for the exercise) ---
tag_values = {
    "{{P_i}}": 3250,     # psi
    "{{P_wh}}": 820,     # psi
    "{{P_wf}}": 1450,    # psi
    "{{Q_o}}": 1250,     # bbl/d
    "{{Q_g}}": 450,      # Mscf/d
    "{{WC}}": 28,        # %
    "{{GOR}}": 850,      # scf/bbl
    "{{API}}": 32,       # deg API
    "{{T_res}}": 92,     # °C
    "{{k}}": 120         # mD
}

def replace_in_paragraph(paragraph, mapping):
    """
    Replace all tags found in a paragraph, even if split across runs.
    Strategy: rebuild paragraph text once, then reset runs to a single run.
    """
    text = paragraph.text
    for k, v in mapping.items():
        text = text.replace(k, str(v))
    # Clear all runs
    for _ in range(len(paragraph.runs)):
        paragraph.runs[0].clear()
        paragraph._p.remove(paragraph.runs[0]._r)  # remove run oxml
    # Create a single run with the replaced text
    run = paragraph.add_run(text)

def iter_block_items(parent):
    """
    Yield paragraphs and tables in document order.
    This covers Document and _Cell containers.
    """
    if isinstance(parent, Document):
        parent_elm = parent.element.body
    elif isinstance(parent, _Cell):
        parent_elm = parent._tc
    else:
        return
    for child in parent_elm.iterchildren():
        if isinstance(child, CT_P):
            yield parent.paragraphs[0] if isinstance(parent, Document) and len(parent.paragraphs) > 0 and child is parent.element.body.p_lst[0] else parent.paragraphs[parent.element.body.p_lst.index(child)] if isinstance(parent, Document) else _paragraph_from_child(parent, child)
        else:
            # Table
            for t in parent.tables:
                if t._tbl is child:
                    yield t

def _paragraph_from_child(container, child):
    """
    Find the paragraph object matching a CT_P 'child' inside a container (_Cell).
    """
    for p in container.paragraphs:
        if p._p is child:
            return p
    # Fallback: create a new paragraph reference if not found
    return container.add_paragraph(child.text)

def replace_in_table(table, mapping):
    """
    Replace tags in all cells of a table.
    """
    for row in table.rows:
        for cell in row.cells:
            for para in cell.paragraphs:
                replace_in_paragraph(para, mapping)

# --- Process document ---
doc = Document(input_path)

# Replace in top-level paragraphs and in tables
for para in doc.paragraphs:
    replace_in_paragraph(para, tag_values)

for tbl in doc.tables:
    replace_in_table(tbl, tag_values)

# Also replace inside nested tables within cells (defensive)
for tbl in doc.tables:
    for row in tbl.rows:
        for cell in row.cells:
            for nested_tbl in cell.tables:
                replace_in_table(nested_tbl, tag_values)

doc.save(output_path)
print("Saved:", output_path)


Saved: /content/Petro_Tags_Filled.docx
